In [1]:
!pip install arxiv

  Using cached arxiv-2.1.3-py3-none-any.whl.metadata (6.1 kB)
  Using cached feedparser-6.0.11-py3-none-any.whl.metadata (2.4 kB)
  Using cached sgmllib3k-1.0.0-py3-none-any.whl
Using cached arxiv-2.1.3-py3-none-any.whl (11 kB)
Using cached feedparser-6.0.11-py3-none-any.whl (81 kB)


In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper



In [11]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki=WikipediaQueryRun(
    api_wrapper=api_wrapper
)


In [12]:
wiki.name

'wikipedia'

In [ ]:
# custom tool 

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


loader=WebBaseLoader("https://docs.smith.langchain.com/")

docs=loader.load()

documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=250).split_documents(docs)

vector_db=FAISS.from_documents(documents,OpenAIEmbeddings())

retriever=vector_db.as_retriever()
 
retriever



VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x1220ab250>, search_kwargs={})

In [7]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool=create_retriever_tool(retriever,"LangSmith",
                        "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")



In [9]:
retriever_tool.name

'LangSmith'

In [10]:
# Arxiv Tool 

from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun


arxiv_wrapper=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)

arxiv.name

'arxiv'

In [15]:
tools=[wiki,arxiv,retriever_tool]

In [16]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/Users/bhanuprasadchetla/Desktop/aiml/Langchain/venv/lib/python3.11/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='LangSmith', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x11278c5e0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchai

In [17]:
from dotenv import load_dotenv
import os
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")

from langchain_openai import ChatOpenAI

llm=ChatOpenAI(model="gpt-3.5-turbo-0125",temperature=0)


In [19]:
from langchain import hub

prompt=hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [20]:
# Agents, language model is used as a resoning engine to determine which action to take in which order

from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)


In [21]:
# agents executer

from langchain.agents import AgentExecutor

agent_executor=AgentExecutor(agent=agent,
                             tools=tools,
                             verbose=True)

In [22]:
agent_executor.invoke({"input":"Tell me about LangSmith"})




> Entering new AgentExecutor chain...

Invoking: `LangSmith` with `{'query': 'LangSmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

LangSmith + LangChain OSSLangSmith integrates seamlessly with LangChain's open source frameworks langchain and langgraph, with no extra instrumentation needed.If you're already using either of these, see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with LangGraph.
Observability​
Observability is important for any software application, but especially so for LLM applications. LLMs are non-deterministic by nature, meaning they can produce unexpected results. This makes them trickier than normal to debug.
This is where LangSmith can help! LangSmith has LLM-native observability, allowing you to get meaningful insights from your application. LangSmith’s observability features have you covered throughout all stages of application development - from prototyping, to beta testing, to production.

Skip to main contentJoin u

{'input': 'Tell me about LangSmith',
 'output': 'LangSmith is a platform for building production-grade Large Language Model (LLM) applications. It allows you to closely monitor and evaluate your application, enabling you to ship quickly and with confidence. LangSmith provides observability features, evaluation capabilities, and tools for prompt engineering to help you optimize and improve your LLM applications.\n\nObservability:\n- Analyze traces in LangSmith and configure metrics, dashboards, and alerts based on these traces.\n- Monitor and evaluate your application throughout all stages of development, from prototyping to production.\n\nEvaluations:\n- Evaluate your application over production traffic to score application performance and receive human feedback on your data.\n- Use the LangSmith SDK and UI to build and run high-quality evaluations easily.\n- Analyze evaluation results in the LangSmith UI and compare results over time.\n- Collect human feedback on your data to enhance 

In [23]:
agent_executor.invoke({"input":"Tell me about Machine Learning"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Machine Learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks witMachine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data. ML algorithms enable computers to learn and improve from experience without being explicitly programmed. This allows them to perform tasks such as data analysis, pattern recognition, and decision-making.

> Finished chain.


{'input': 'Tell me about Machine Learning',
 'output': 'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data. ML algorithms enable computers to learn and improve from experience without being explicitly programmed. This allows them to perform tasks such as data analysis, pattern recognition, and decision-making.'}

In [24]:
agent_executor.invoke({"input":"What's the paper 1605.08386 about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1605.08386'}`


Published: 2016-05-26
Title: Heat-bath random walks with Markov bases
Authors: Caprice Stanley, Tobias Windisch
Summary: Graphs on lattice points are studied whose edges come from a finite set of
allowed moves of arbitrary length. We show that the diThe paper with identifier 1605.08386 is titled "Heat-bath random walks with Markov bases" by Caprice Stanley and Tobias Windisch. It discusses the study of graphs on lattice points where the edges are derived from a finite set of allowed moves of arbitrary length. The paper explores the properties of these graphs and their connections to Markov bases in the context of heat-bath random walks.

> Finished chain.


{'input': "What's the paper 1605.08386 about?",
 'output': 'The paper with identifier 1605.08386 is titled "Heat-bath random walks with Markov bases" by Caprice Stanley and Tobias Windisch. It discusses the study of graphs on lattice points where the edges are derived from a finite set of allowed moves of arbitrary length. The paper explores the properties of these graphs and their connections to Markov bases in the context of heat-bath random walks.'}